In [ ]:
def parse_ibrecords(data_array):

    data_list = []

    for obj in data_array:
        data = {}

        if hasattr(obj, "contract"):
            util.logging.debug(obj.contract)
            contract = util.dataclassNonDefaults(obj.contract)
            data = {**data, **contract}

        if hasattr(obj, "order"):
            util.logging.debug(obj.order)
            order = util.dataclassNonDefaults(obj.order)
            order.pop("softDollarTier")
            data = {**data, **order}

        if hasattr(obj, "orderStatus"):
            util.logging.debug(obj.orderStatus)
            orderStatus = util.dataclassNonDefaults(obj.orderStatus)
            data = {**data, **orderStatus}

        if hasattr(obj, "fills"):
            util.logging.debug(obj.fills)
            fills = {"fills": obj.fills}
            data = {**data, **fills}

        if hasattr(obj, "log"):
            util.logging.debug(obj.log)
            logs = {"log": [util.dataclassAsDict(e) for e in obj.log]}
            data = {**data, **logs}

        if hasattr(obj, "advancedError"):
            util.logging.debug(obj.advancedError)
            advancedError = {"advancedError": obj.advancedError}
            data = {**data, **advancedError}

        if type(obj) == Order:
            util.logging.debug(obj)
            order = util.dataclassNonDefaults(obj)
            order.pop("softDollarTier")
            data = {**data, **order}

        data_list.append(data)

    return data_list


def print_ibrecords_table(
    data_array,
    cols=[
        "localSymbol",
        "permId",
        "status",
        "orderType",
        "action",
        "lmtPrice",
        "remaining",
    ],
):
    df = pd.DataFrame(data_array)
    df = df[cols]
    print(df)


# openTrades = parse_ibrecords(ib.openTrades())
accountSummary = ib.accountSummary()
accountValues = ib.accountValues()

openTrades = parse_ibrecords(ib.openTrades())
# util.df([d for d in openTrades])

openOrders = parse_ibrecords(ib.openOrders())
# util.df([d for d in openOrders])

trades = parse_ibrecords(ib.trades())
# util.df([d for d in trades])

orders = parse_ibrecords(ib.orders())
# util.df([d for d in orders])


------- U10394496 ----------
	 Cushion 0.645628
	 LookAheadExcessLiquidity 47858.58
	 NetLiquidation 95132.47
	 TotalCashValue 93063.82
------- U2340948 ----------
	 Cushion 1
	 FullExcessLiquidity 343.82
	 LookAheadExcessLiquidity 343.82
	 NetLiquidation 91511.92


In [33]:
# NQM2024 contract
print("Setting contract to NQM2024 symbol / JUN 2024 / Contract(conId=620730920)")
contract = Contract(conId=620730920)
ib.qualifyContracts(contract)

ticker = ib.reqMktDepth(contract)

ib.sleep(5)


Setting contract to NQM2024 symbol / JUN 2024 / Contract(conId=620730920)
     6.0    18900.5 | 18901.0    9.0     
    16.0   18900.25 | 18901.25   7.0     
    17.0    18900.0 | 18901.5    10.0    
    14.0   18899.75 | 18901.75   11.0    
    11.0    18899.5 | 18902.0    10.0    


In [ ]:
import sqlite3
from datetime import datetime
from typing import Dict, List, Any, Optional


class SQLiteWrapper:
    def __init__(self, db_name: str, drop_tables=[]):
        self.conn = sqlite3.connect(db_name)
        self.drop_tables(drop_tables)
        self.create_trades_table()

    def drop_tables(self, drop_tables: List[str]):
        for tbl in drop_tables:
            drop_table_sql = f"DROP TABLE IF EXISTS {tbl};"
            self.conn.execute(drop_table_sql)
            self.conn.commit()

    def create_trades_table(self):

        create_table_sql = """
        CREATE TABLE IF NOT EXISTS trades (
            secType TEXT,
            conId INTEGER,
            symbol TEXT,
            lastTradeDateOrContractMonth TEXT,
            right TEXT,
            multiplier TEXT,
            exchange TEXT,
            currency TEXT,
            localSymbol TEXT,
            tradingClass TEXT,
            orderId INTEGER,
            clientId INTEGER,
            permId INTEGER PRIMARY KEY,
            action TEXT,
            totalQuantity REAL,
            filledQuantity REAL,
            orderType TEXT,
            lmtPrice REAL,
            auxPrice REAL,
            trailStopPrice REAL,
            strike REAL,
            tif TEXT,
            parentPermId INTEGER,
            ocaType INTEGER,
            displaySize INTEGER,
            rule80A TEXT,
            openClose TEXT,
            volatilityType INTEGER,
            deltaNeutralOrderType TEXT,
            shareholder TEXT,
            referencePriceType INTEGER,
            refFuturesConId INTEGER,
            account TEXT,
            clearingIntent TEXT,
            autoCancelDate TEXT,
            adjustedOrderType TEXT,
            cashQty REAL,
            dontUseAutoPriceForHedge INTEGER,
            fills TEXT,
            log TEXT,
            advancedError TEXT,
            status TEXT,
            remaining REAL,
            ocaGroup TEXT
        );
        """
        self.conn.execute(create_table_sql)
        self.conn.commit()

    def upsert_trades_record(self, data: Dict[str, Any]):
        # Convert datetime objects in log to string
        log_entries = data.get("log", [])
        for entry in log_entries:
            entry["time"] = entry["time"].isoformat()
        data["log"] = str(log_entries)

        # Convert fills list to string
        data["fills"] = str(data.get("fills", []))

        # Handle boolean conversion
        data["dontUseAutoPriceForHedge"] = int(data["dontUseAutoPriceForHedge"])

        keys = ", ".join(data.keys())
        question_marks = ", ".join("?" for _ in data)
        update_clause = ", ".join(f"{key}=excluded.{key}" for key in data)

        upsert_sql = f"""
        INSERT INTO trades ({keys})
        VALUES ({question_marks})
        ON CONFLICT(permId) DO UPDATE SET
        {update_clause};
        """

        values = tuple(data.values())
        self.conn.execute(upsert_sql, values)
        self.conn.commit()

    def read_record(self, permId: int) -> Optional[Dict[str, Any]]:
        select_sql = "SELECT * FROM orders WHERE permId = ?;"
        cursor = self.conn.execute(select_sql, (permId,))
        row = cursor.fetchone()
        if row:
            col_names = [description[0] for description in cursor.description]
            record = dict(zip(col_names, row))
            # Convert log string back to list of dicts
            record["log"] = eval(record["log"])
            # Convert fills string back to list
            record["fills"] = eval(record["fills"])
            # Convert integer back to boolean
            record["dontUseAutoPriceForHedge"] = bool(
                record["dontUseAutoPriceForHedge"]
            )
            return record
        return None

    def close(self):
        self.conn.close()


def parse_OpenTrades(ib: IB):
    trades = ib.trades()

    data = None
    data_list = []

    for trade in trades:
        contract = util.dataclassNonDefaults(trade.contract)
        order = util.dataclassNonDefaults(trade.order)
        order.pop("softDollarTier")
        orderStatus = util.dataclassNonDefaults(trade.orderStatus)
        order["fills"] = trade.fills
        order["log"] = [util.dataclassAsDict(e) for e in trade.log]
        order["advancedError"] = trade.advancedError

        util.logging.debug(f"Processing trade with orderId: {order['permId']}")

        data = {**contract, **order, **orderStatus}

        data_list.append(data)

    return data_list


def insert_OpenTrades(ib: IB, db: SQLiteWrapper):
    data_list = parse_OpenTrades(ib)

    for data in data_list:
        try:
            db.upsert_trades_record(data)

        except Exception as e:
            util.logging.debug(f"Error upserting record: {e}")
            util.logging.debug(data)

    return True


def test_trades_upsert():
    # db = SQLiteWrapper('test.db', drop_tables = ['trades', 'orders'])
    db = SQLiteWrapper("test.db", drop_tables=["trades"])

    insert_OpenTrades(ib, db)

    db.close()

    return True


def test_read_record():
    db = SQLiteWrapper("test.db")
    record = db.read_record(262255346)
    util.logging.debug(record)
    db.close()
    return True


# Example usage
if __name__ == "__main__":
    test_trades_upsert()
